# Prospecção de Dados (Data Mining) DI/FCUL - HA2

## Second Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP:`02`

* João Martins, 62532 - Hours worked on the project: 8k
* Rúben Torres, 62531 - Hours worked on the project: 8k
* Nuno Pereira, 56933 - Hours worked on the project: 8k



## Objectives
The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Process and identify the most relevant rules

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**


### 1. Read the Dataset

Dataset: Instacart [Market Basket Analysis](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis)



The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Identify the most relevant rules

Please download the HA files from this moodle folder and inspect it like this:

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

The dataset has been preprocessed with transactions as lists of integers. It is necessary a decoder to see the actual products bought together

Uncompress the data set (zip file) which will produce 2 files:
* `order_products.pickle` a Python pickle with all the trasactions
* A `products.txt` - decoder table with all the data relative to each item



In [ ]:
#Read product names and IDs
lines=open("products.txt", "rt", encoding="utf8").readlines()
products=[0]*len(lines)
for lin in lines[1:]:
    pid, pname, aid, did=lin.strip().split("\t")    
    products[int(pid)]=pname
    
#read transactions
import pickle
orders=pickle.load(open("order_products.pickle", "rb"))

#check names of products on transaction 2 (example):
print("Transaction 2 is", orders[2])
for prod in orders[2]: print("Code: %5d is product: %s" %(prod, products[prod]))

### Objective 1 - Analyze the itemset/rules generation procedure

1. From the approaches used in classes make a performance analysis up to a threshold level of support
2. Define a good support threshold for analysis according to your computational capabilities


In [ ]:
### Add supporting functions here
from scipy.sparse import csr_matrix
from IPython.display import display_html
from itertools import chain,cycle

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from pyfim import pyeclat
from PD_freqitems import freqitemsets

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time


def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

def plot_execution_time(D):
    plt.plot(np.log(D["num_itemsets"]), np.log(D["apriori"]), c="r", label="apriori")
    plt.plot(np.log(D["num_itemsets"]), np.log(D["FP-growth"]), c="k", label="fp-growth")
    plt.plot(np.log(D["num_itemsets"]), np.log(D["ECLAT"]), c="g", label="eclat")
    plt.plot(np.log(D["num_itemsets"]), np.log(D["PD"]), c="b", label="PD")
    plt.grid()
    plt.legend()
    plt.show()



################################################################
#Compute binary database
tr_enc = TransactionEncoder()
trans_array = tr_enc.fit_transform(orders.values(), sparse=True)
trans_array = csr_matrix(trans_array, shape=(len(orders.values()), len(tr_enc.columns_)))

products_list = []
for i in range(len(tr_enc.columns_)+1):
    products_list.append(products[i])

products_list.pop(0)
binary_database = pd.DataFrame.sparse.from_spmatrix(trans_array, columns=products_list)


########################################################################################################
# MemoryError: Unable to allocate 149. GiB for an array with shape (3214874, 49677) and data type bool #
########################################################################################################

# tr_enc = TransactionEncoder()
# trans_array = tr_enc.fit(orders.values()).transform(orders.values())
# products_list = []
# for i in range(len(tr_enc.columns_)+1):
#     products_list.append(products[i])
# binary_database = pd.DataFrame(trans_array, columns=tr_enc.columns_)
# binary_database

################################################################

binary_database

In [ ]:
### Add processing code here

# FI_apriori = apriori(binary_database, min_support=0.01, use_colnames=True)
# FI_apriori_len = FI_apriori.copy()

# FI_apriori_len['length'] = FI_apriori_len['itemsets'].apply(lambda x: len(x))
# FI_apriori_len = FI_apriori_len.sort_values(by=['length'], ascending=False)


FI_fpg=fpgrowth(binary_database, min_support=0.01, use_colnames=True)
FI_fpg

FI_fpg['length'] = FI_fpg['itemsets'].apply(lambda x: len(x))
FI_fpg.sort_values(by=['length'], ascending=False)

FI_eclat= pyeclat(orders.values(), 0.01)
FI_eclat

FI_FI_fpg_filtered = FI_fpg[ (FI_fpg['support'] >= 0.01) & (FI_fpg['length'] >= 2) ]
FI_FI_fpg_filtered

# display_side_by_side(FI_fpg, FI_eclat)




D={"threshold": [0.05, 0.02, 0.015], # 0.01
"num_itemsets": [],
"apriori":[],
"FP-growth": [],
"ECLAT": [],
"PD": []}

#WARNING! 0.01 sometimes crashes my computer it may well crash yours!
for min_supp in D["threshold"]:
    t0 = time.time()
    FI_apriori=apriori(binary_database, min_supp)
    t1 = time.time()
    D["num_itemsets"].append(FI_apriori.shape[0])
    D["apriori"].append(t1-t0)

    FI_fpg= fpgrowth(binary_database, min_supp)
    t2 = time.time()

    D["FP-growth"].append(t2-t1)

    FI_eclat= pyeclat(orders.values(), min_supp)
    t3 = time.time()
    D["ECLAT"].append(t3-t2)

    FI_pdfis= freqitemsets(orders.values(), min_supp)
    t4 = time.time()
    D["PD"].append(t4-t3)

    print(min_supp, FI_apriori.shape[0],"\n\tApriori time:", t1-t0,
                                        "\n\tFP-growth time:", t2-t1,
                                        "\n\tECLAT time:", t3-t2,
                                        "\n\tPD time:", t4-t3)

df_performance=pd.DataFrame(D)
df_performance

plot_execution_time(D)

### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum

### Objective 2 - Identify the most relevant rules

1. From your predefined support level generate all available itemsets and generate rules
2. Identify a set of 10 relevant rules using the Highest Lift criterion
    * **NOTE**: Present the rules with the product names and not as Integers 
3. Identify the Maximal and Closed Itemsets for the same level of support and generate 5 rules using the Highest Lift
    * **NOTE**: Do not list the Maximal or Closed Itemsets. Present just a few as an example and mention **how many** Closed and Maximal Itemsets were found for the selected support level
    * **NOTE**: Present the rules with the product names and not as Integers 


In [ ]:
### Add supporting functions here

from mlxtend.frequent_patterns import association_rules, fpmax

all_rules = association_rules(FI_fpg, metric="confidence", min_threshold=0.05)
all_rules



FI_fpmax=fpmax(binary_database, min_support=0.01, use_colnames=True)
FI_fpmax



Trans_maximal = pyeclat(orders.values(), 0.01, target="m")
print("Number of Maximal Itemsets", len(Trans_maximal) )
Trans_maximal


Trans_closed = pyeclat(binary_database, 0.01, target="c")
print("Number of Closed Itemsets", len(Trans_closed))
Trans_closed

# display_side_by_side(all_rules, FI_fpmax, Trans_maximal, Trans_closed)


In [ ]:
all_rules

In [ ]:
FI_fpmax

In [ ]:
Trans_maximal

In [ ]:
Trans_closed

In [ ]:
### Add processing code here



### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum
